In [ ]:
import numpy as np
import pandas as pd 
from pykrige.ok import OrdinaryKriging
from pykrige.kriging_tools import write_asc_grid
import pykrige.kriging_tools as kt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import LeaveOneOut,LeavePOut

#import floor plans
img = mpimg.imread("floor_plan.png") 
img2 = mpimg.imread("floor_plan2.png")
img3 = mpimg.imread("floor_plan3.png")

#read coordinates of sensors 
position=pd.read_csv('sensor_position.txt',delimiter="\t")
x=np.array(position['X']) 
y=np.array(position['Y']) 
sensor=np.array(position['Sensor'])

#read temperature and humitidy data
temperature=pd.read_csv('data_temperature.txt',delimiter="\t",header = None)
data_temperature=np.array(temperature[:])
humidity=pd.read_csv('data_humidity.txt',delimiter="\t",header = None)
data_humidity=np.array(humidity[:])

#create a grid in order to run the kriging in it
grid_space = 0.1
grid_x = np.arange(0, 55.1, grid_space)  
grid_y = np.arange(0, 145.1, grid_space)

#various variogram models
var_model=['spherical', 'exponential', 'gaussian', 'linear']

In [ ]:
#kriging for temperature estimation and plotting: estimate for each time/spherical variogram model
for i in range(len(data_temperature)):
    OK = OrdinaryKriging(x, y, data_temperature[i], variogram_model=var_model[0], verbose=True, enable_plotting=True,nlags=20)
    z_estim, e_estim = OK.execute('grid', grid_x, grid_y)

    xintrp, yintrp = np.meshgrid(grid_x, grid_y)
    
    fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(10, 5))
    
    ax1.set_title('Estimated temperature values\n')
    ax1.scatter(x, y,s=10,c='black' ,zorder=2,marker='o',label='Sensor')
    ax1.imshow(img2, extent=[0, 55, 145, 0],zorder=3)
    contour1=ax1.contourf(xintrp, yintrp, z_estim, np.linspace(15, 30,400,endpoint=True) ,cmap='jet', zorder=1)
    cb1=plt.colorbar(contour1,ax=ax1,ticks=np.linspace(15, 30,16,endpoint=True))
 
    ax2.set_title('Error of estimated temperature values\n')
    ax2.scatter(x, y,s=10,c='black' ,zorder=2,marker='o',label='Sensor')
    ax2.imshow(img2, extent=[0, 55, 145, 0],zorder=3)
    contour2=ax2.contourf(xintrp, yintrp, e_estim , np.linspace(0, 1.0,100,endpoint=True) ,cmap='Greys', zorder=1) 
    cb2=plt.colorbar(contour2,ax=ax2,ticks=np.linspace(0, 1.0,10,endpoint=True))
    
    plt.show()

In [ ]:
#leave one out method(loo) for cross validation kriging temperature
loo = LeaveOneOut()

#perfom loo for each variogram model
for model in var_model:
    print("\nVariogram model: {}\n".format(model))
    
    for i in range(len(data_temperature)):
        error = np.empty((0,2))
        data = np.stack((x,y,data_temperature[i],sensor),axis=-1)
        
#         print('{:<30s}{:>30s}'.format('Sensor left out for test', 'Difference between real and estimated value in °C'))
        
        for train_id, test_id in loo.split(data):
            x_train = data[train_id,:]
            coords_test = data[test_id, [0,1]]
            z_test = data[test_id, 2]
            sensor_test=data[test_id,3]
        
            OK = OrdinaryKriging(x_train[:,0], x_train[:,1], x_train[:,2], variogram_model=model, verbose=False, enable_plotting=False,nlags=20)
            z_estim, e_estim = OK.execute("points", coords_test[0], coords_test[1])
            
            diff=np.array([abs(z_estim[0] - z_test[0])])
            
#             print('{}\t\t {}'.format(sensor_test, diff))
            
            error=np.append(error,np.hstack(([diff],[sensor_test])),axis=0)
        
        print("Avarage error of all possible training groups: {} °C".format(np.mean(error[:,0])))

In [ ]:
#leave p out method (lpo) for cross validation kriging temperature (2<p<7)
for p in range(2,8,1):
    
    lpo = LeavePOut(p)
    print("\nVariogram model: {}\nLeave {} out method\n".format(var_model[0],p))
    
    for i in range(len(data_temperature)):
        error = np.empty((0,2*p))
        data = np.stack((x,y,data_temperature[i],sensor),axis=-1)
        
#         print('{:<30s}{:>30s}'.format('Sensor left out for test', 'Difference between real and estimated value in °C'))
        
        for train_id, test_id in lpo.split(data):
            x_train = data[train_id,:]
            coords_test = data[test_id, :]
            z_test = data[test_id, 2]
            sensor_test=data[test_id,3]
        
            OK = OrdinaryKriging(x_train[:,0], x_train[:,1], x_train[:,2], variogram_model=var_model[0], verbose=False, enable_plotting=False,nlags=20)
            z_estim, e_estim = OK.execute("points", coords_test[:,0], coords_test[:,1])
            
            diff=np.array([abs(z_estim[:] - z_test[:])])
            
#             print('{}\t\t {}'.format(sensor_test, diff))
            
            error=np.append(error,np.hstack((diff[:],[sensor_test])),axis=0)
        
        print("Avarage error of all possible training groups: {} °C".format(np.mean(error[:,0:(p-1)])))

In [ ]:
#kriging for relative humidity estimation and plotting: estimate for each time/spherical variogram model
for i in range(len(data_humidity)):
    OK = OrdinaryKriging(x, y, data_humidity[i], variogram_model=var_model[0], verbose=True, enable_plotting=True,nlags=20)
    z_estim, e_estim = OK.execute('grid', grid_x, grid_y)

    xintrp, yintrp = np.meshgrid(grid_x, grid_y)
    
    fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(10, 5))
    
    ax1.set_title('Estimated humidity values\n')
    ax1.scatter(x, y,s=10,c='black' ,zorder=2,marker='o',label='Sensor')
    ax1.imshow(img2, extent=[0, 55, 145, 0],zorder=3)
    contour1=ax1.contourf(xintrp, yintrp, z_estim, np.linspace(30, 60,100,endpoint=True) ,cmap='jet', zorder=1)
    cb1=plt.colorbar(contour1,ax=ax1,ticks=np.linspace(30, 60,4,endpoint=True))
 
    ax2.set_title('Variance of estimated humidity values\n')
    ax2.scatter(x, y,s=10,c='black' ,zorder=2,marker='o',label='Sensor')
    ax2.imshow(img2, extent=[0, 55, 145, 0],zorder=3)
    contour2=ax2.contourf(xintrp, yintrp, e_estim , np.linspace(0, 15,100,endpoint=True) ,cmap='Greys', zorder=1) 
    cb2=plt.colorbar(contour2,ax=ax2,ticks=np.linspace(0, 15,16,endpoint=True))
    
    plt.show()

In [ ]:
#leave one out method(loo) for cross validation kriging humidity
loo = LeaveOneOut()

#perfom loo for each variogram model
for model in var_model:
    print("\nVariogram model: {}\n".format(model))
    
    for i in range(len(data_humidity)):
        error = np.empty((0,2))
        data = np.stack((x,y,data_humidity[i],sensor),axis=-1)
        
        print('{:<30s}{:>30s}'.format('Sensor left out for test', 'Difference between real and estimated value in %'))
        
        for train_id, test_id in loo.split(data):
            x_train = data[train_id,:]
            coords_test = data[test_id, [0,1]]
            z_test = data[test_id, 2]
            sensor_test=data[test_id,3]
        
            OK = OrdinaryKriging(x_train[:,0], x_train[:,1], x_train[:,2], variogram_model=model, verbose=False, enable_plotting=False,nlags=20)
            z_estim, e_estim = OK.execute("points", coords_test[0], coords_test[1])
            
            diff=np.array([abs(z_estim[0] - z_test[0])])
            
            print('{}\t\t {}'.format(sensor_test, diff))
            
            error=np.append(error,np.hstack(([diff],[sensor_test])),axis=0)
        
        print("Avarage error of all possible training groups: {} %".format(np.mean(error[:,0])))

In [ ]:
#remove multisensor 27 position and values
xnew=np.delete(x,7,0) 
ynew=np.delete(y,7,0)
sensornew=np.delete(sensor,7,0)
data_humiditynew=np.delete(data_humidity,7,1)

#loo once again
for model in var_model:
    print("\nVariogram model: {}\n".format(model))
    
    for i in range(len(data_humidity)):
        error = np.empty((0,2))
        data = np.stack((xnew,ynew,data_humiditynew[i],sensornew),axis=-1)
        
        print('{:<30s}{:>30s}'.format('Sensor left out for test', 'Difference between real and estimated value in %'))
        
        for train_id, test_id in loo.split(data):
            x_train = data[train_id,:]
            coords_test = data[test_id, [0,1]]
            z_test = data[test_id, 2]
            sensor_test=data[test_id,3]
        
            OK = OrdinaryKriging(x_train[:,0], x_train[:,1], x_train[:,2], variogram_model=model, verbose=False, enable_plotting=False,nlags=20)
            z_estim, e_estim = OK.execute("points", coords_test[0], coords_test[1])
            
            diff=np.array([abs(z_estim[0] - z_test[0])])
            
            print('{}\t\t {}'.format(sensor_test, diff))
            
            error=np.append(error,np.hstack(([diff],[sensor_test])),axis=0)
        
        print("Avarage error of all possible training groups: {} %".format(np.mean(error[:,0])))